<a href="https://colab.research.google.com/github/m-t-abbasi/SpatialPlanning/blob/main/section6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import geopandas as gpd
import numpy as np
import random

# مسیر فایل شیپ را تعیین می‌کنیم
shapefile_path = "/content/landuse_shp.shp"

# خواندن فایل شیپ
gdf = gpd.read_file(shapefile_path)

# تعداد کل پارسل‌های منطقه
total_parcels = len(gdf)

# شمارش تعداد پارسل‌ها به تفکیک کاربری
landuse_counts = gdf['landuse2'].value_counts().to_dict()

# محاسبه مساحت هر کاربری بر حسب کل مساحت پارسل‌های منطقه
landuse_areas = gdf.groupby('landuse2')['SHAPE_Area'].sum().to_dict()

# تعریف اندازه‌های مختلف بافر
buffer_sizes = [100, 200, 300]

# دیکشنری برای ذخیره میانگین نسبت‌ها بر اساس کاربری و بافر
mean_ratios = {}

# تعداد پارسل‌هایی که برای محاسبه استفاده می‌شود
sample_size = 10

# پیمایش تک تک کاربری‌ها
for landuse_type in landuse_counts.keys():
    mean_ratios[landuse_type] = {}  # دیکشنری برای ذخیره میانگین نسبت‌ها بر اساس بافر
    
    # پیمایش تک تک اندازه‌های بافر
    for buffer_size in buffer_sizes:
        ratios = []  # لیستی برای ذخیره نسبت‌های محاسبه شده
        
        # پیمایش تک تک پارسل‌ها
        for index, row in gdf.iterrows():
            parcel_id = row['OBJECTID']  # شماره پارسل
            
            buffer = row.geometry.buffer(buffer_size)
            
            # پیدا کردن پارسل‌هایی که کاملاً در داخل هر بافر قرار دارند
            parcels_within_buffer = gdf[gdf.geometry.within(buffer)]
            
            # شمارش تعداد پارسل‌ها به تفکیک کاربری
            landuse_counts_within_buffer = parcels_within_buffer['landuse2'].value_counts().to_dict()
            
            # محاسبه مساحت هر کاربری در هر بافر
            landuse_areas_within_buffer = parcels_within_buffer.groupby('landuse2')['SHAPE_Area'].sum().to_dict()
            
            # محاسبه نسبت آماری برای هر کاربری
            S_di = sum(landuse_areas_within_buffer.values())
            N_k = landuse_counts[landuse_type]
            S_k = landuse_areas[landuse_type]
            N = total_parcels
            S = sum(landuse_areas.values())
            
            ratio = ((landuse_counts_within_buffer.get(landuse_type, 0) * landuse_areas_within_buffer.get(landuse_type, 0)) / (N_k * S_di)) / ((N_k * S_k) / (N * S))
            ratios.append(ratio)
    
        # محاسبه میانگین نسبت‌ها برای هر کاربری و بافر
        mean_ratio = np.mean(ratios)
        
        # ذخیره میانگین نسبت در دیکشنری
        mean_ratios[landuse_type][buffer_size] = mean_ratio

# دیکشنری برای ذخیره شاخص غنی شدگی بر اساس کاربری و بافر
enrichment_indices = {}

# پیمایش تک تک کاربری‌ها
for landuse_type in mean_ratios.keys():
    enrichment_indices[landuse_type] = {}  # دیکشنری برای ذخیره شاخص غنی شدگی بر اساس بافر
    
    # پیمایش تک تک اندازه‌های بافر
    for buffer_size in buffer_sizes:
        ratio = mean_ratios[landuse_type][buffer_size]
        
        # محاسبه لگاریتم نسبت
        enrichment_index = np.log(ratio)
        
        # ذخیره شاخص غنی شدگی در دیکشنری
        enrichment_indices[landuse_type][buffer_size] = enrichment_index


# چاپ شاخص غنی شدگی بر اساس کاربری و بافر
for landuse_type in enrichment_indices.keys():
    for buffer_size, enrichment_index in enrichment_indices[landuse_type].items():
        print(f"شاخص غنی شدگی برای کاربری {landuse_type} و بافر {buffer_size}: {enrichment_index}")


شاخص غنی شدگی برای کاربری maskooni و بافر 100: -3.6949416495691154
شاخص غنی شدگی برای کاربری maskooni و بافر 200: -2.546758455922262
شاخص غنی شدگی برای کاربری maskooni و بافر 300: -1.9061152450636727
شاخص غنی شدگی برای کاربری tejari و بافر 100: -2.303323193142576
شاخص غنی شدگی برای کاربری tejari و بافر 200: -1.441810622625696
شاخص غنی شدگی برای کاربری tejari و بافر 300: -0.8801893916328429
شاخص غنی شدگی برای کاربری bayer و بافر 100: -1.6875399543148932
شاخص غنی شدگی برای کاربری bayer و بافر 200: -0.9128417551546545
شاخص غنی شدگی برای کاربری bayer و بافر 300: -0.46501300569073756
شاخص غنی شدگی برای کاربری khadamati و بافر 100: -0.7975987461543872
شاخص غنی شدگی برای کاربری khadamati و بافر 200: 0.33434623000508673
شاخص غنی شدگی برای کاربری khadamati و بافر 300: 1.0778133288415073
شاخص غنی شدگی برای کاربری amozshi و بافر 100: 0.2709720529077686
شاخص غنی شدگی برای کاربری amozshi و بافر 200: 1.4366476603575689
شاخص غنی شدگی برای کاربری amozshi و بافر 300: 2.1304752793943464
شاخص غنی شدگی بر